In [2]:
import warnings
warnings.filterwarnings('ignore')

import sys, os, random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from PIL import Image
import cv2
%matplotlib inline

# tweaks for libraries
np.set_printoptions(precision=4, linewidth=1024, suppress=True)
plt.style.use('seaborn')
sns.set_style('darkgrid')
sns.set(context='notebook',font_scale=1.10)

# Pytorch imports
import torch
gpu_available = torch.cuda.is_available()
print('Using Pytorch version %s. GPU %s available' % (torch.__version__, "IS" if gpu_available else "IS **NOT**"))
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch import optim
from torchsummary import summary

# import the Pytorch Toolkit here....
import pytorch_toolkit as pytk

# to ensure that you get consistent results across various machines
# @see: https://discuss.pytorch.org/t/reproducibility-over-different-machines/63047
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = True

Using Pytorch version 1.10.2. GPU IS **NOT** available


In [3]:
# download an image
!wget https://www.dropbox.com/s/l98leemr7r5stnm/Hemanvi.jpeg -o ./images/Hemanvi.jpeg 

In [4]:
img = cv2.imread('./images/Hemanvi.jpeg')
img2 = img[50:250, 40:240]
# show original image
img_color = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
plt.imshow(img_color);
plt.show()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
img_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
plt.imshow(img_gray, cmap='gray');
# plt.axis("off")
# plt.grid(False)
plt.show()

In [ ]:
img_gray_small = cv2.resize(img_gray, (32,32))
plt.imshow(img_gray_small, cmap='gray');

In [ ]:
data_dir = './data'
# download the training & test Fashion MNIST datasets into ./data folder
xforms = transforms.Compose([transforms.ToTensor(),])
train_dataset = datasets.FashionMNIST(data_dir, download=True, train=True, transform=xforms)
test_dataset = datasets.FashionMNIST(data_dir, download=True, train=False, transform=xforms)
print(f"Downloaded {len(train_dataset)} training records and {len(test_dataset)} testing records")

In [ ]:
# define our class
from torch.optim import SGD

net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 1000),
    nn.ReLU(),
    nn.Linear(1000, 10)
)
model = pytk.PytkModuleWrapper(net)
optimizer = SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])
print(model.summary((1, 28, 28)))

In [ ]:
hist = model.fit_dataset(train_dataset, validation_split=0.20, epochs=10, batch_size=32)

In [ ]:
pytk.show_plots(hist, metric='accuracy', plot_title='Model Performance')

In [ ]:
del model
model = pytk.PytkModuleWrapper(net)
optimizer = SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])
print(model.summary((1, 28, 28)))

In [ ]:
hist = model.fit_dataset(train_dataset, validation_split=0.20, epochs=10, batch_size=10000)

In [ ]:
pytk.show_plots(hist, metric='accuracy', plot_title='Model Performance')